# Evaluate Model

## Access credentials file

In [5]:
import sys, os

In [6]:
def add_path_to_sys_path(path_to_append):
    if not (any(path_to_append in paths for paths in sys.path)):
        sys.path.append(path_to_append)

In [7]:
credentials_path = './../not_shared'
credentials_file_name = os.path.join(credentials_path, 'get_data_access_secrets.py')

In [8]:
add_path_to_sys_path(credentials_path)         
import get_data_access_secrets

# Set up workspace


In [9]:
#Create config to workspace

import azureml.core
from azureml.core import Experiment, Run, Workspace


# Check core SDK version number
print("This notebook was created using version 1.0.8 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

This notebook was created using version 1.0.8 of the Azure ML SDK
You are currently using version 1.0.8 of the Azure ML SDK


In [10]:
subscription_id, resource_group, workspace_name, workspace_region = get_data_access_secrets.get_subscription_info()

In [11]:
#new workspace
ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,
                      create_resource_group = True,
                      exist_ok = True)
ws.get_details()

# ws = Workspace.from_config()
# print('Workspace name: ' + ws.name, 
#       'Azure region: ' + ws.location, 
#       'Subscription id: ' + ws.subscription_id, 
#       'Resource group: ' + ws.resource_group, sep='\n')

{'id': '/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/ghiordanchestxray03rsg/providers/Microsoft.MachineLearningServices/workspaces/sdk-chest-xray',
 'name': 'sdk-chest-xray',
 'location': 'eastus',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'workspaceid': 'fa6444a9-7aaa-47e7-be2f-4f59b6368ef7',
 'description': '',
 'friendlyName': 'sdk-chest-xray',
 'creationTime': '2019-01-25T17:08:07.1455099+00:00',
 'containerRegistry': '/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/ghiordanchestxray03rsg/providers/microsoft.containerregistry/registries/sdkchestacrtufoedhv',
 'keyVault': '/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/ghiordanchestxray03rsg/providers/microsoft.keyvault/vaults/sdkchestkeyvaultvychezka',
 'applicationInsights': '/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/ghiordanchestxray03rsg/providers/microsoft.insights/components/sdkchestinsightsqegtsmwh',
 'identityPrincipalId': '

In [5]:
ws.write_config()

Wrote the config file config.json to: /datadrive01/amlSDKAzureChestXray/code/aml_config/config.json


In [6]:
experiment_name = "chestxray-evaluate"

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

print(exp)

Experiment(Name: chestxray-evaluate,
Workspace: sdk-chest-xray)


In [7]:
from azureml.core import Datastore

In [8]:
datastore_name, container_name, account_name, account_key = get_data_access_secrets.get_blob_credentials()
# ds = Datastore.register_azure_blob_container(workspace=ws, datastore_name = datastore_name, container_name = container_name, account_name = account_name, account_key = account_key)
ds = Datastore.get(workspace=ws, datastore_name=datastore_name)

In [9]:
print(ds.name, ds.datastore_type, ds.account_name, ds.container_name)

chestxraydatastore AzureBlob chestxraystorage xraycontainer


## Allocate Compute

In [10]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your GPU cluster
gpu_cluster_name = "gpucluster"

# Verify that cluster does not exist already
try:
    gpu_cluster = ComputeTarget(workspace=ws, name=gpu_cluster_name)
    print("Found existing gpu cluster")
except ComputeTargetException:
    print("Creating new gpucluster")
    
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_NC24s_v3",
                                                           min_nodes=0,
                                                           max_nodes=1)
    # Create the cluster with the specified name and configuration
    gpu_cluster = ComputeTarget.create(ws, gpu_cluster_name, compute_config)

    # Wait for the cluster to complete, show the output log
    gpu_cluster.wait_for_completion(show_output=True)
    
     # For a more detailed view of current AmlCompute status, use the 'status' property    
#     print(compute_target.status.serialize())

Found existing gpu cluster


## Create Scoring Script

In [11]:
script_folder = './xray_scripts'
eval_script_filename = 'eval.py'

In [25]:
%%writefile {os.path.join(script_folder, eval_script_filename)}

import sys, os

import src.azure_chestxray_utils as azure_chestxray_utils
import src.azure_chestxray_keras_utils as azure_chestxray_keras_utils


parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder')
args = parser.parse_args()
print(args)
print('Data folder is at:', args.data_folder)
base_dir = args.data_folder

import imgaug as ia
from imgaug import augmenters as iaa
ia.seed(1)

import cv2
import keras.backend as K
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, Callback, ModelCheckpoint
import numpy as np
import pandas as pd
import pickle
from keras_contrib.applications.densenet import DenseNetImageNet121
from keras.layers import Dense
from keras.models import Model
from keras.utils import multi_gpu_model
import keras_contrib
from tensorflow.python.client import device_lib
import warnings
from keras.utils import Sequence
import tensorflow as tf
from sklearn import metrics

def get_available_gpus():
    """
    Returns: number of GPUs available in the system
    """
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']


# generator for train and validation data
# use the Sequence class per issue https://github.com/keras-team/keras/issues/1638
class DataGenSequence(Sequence):
    def __init__(self, labels, image_file_index, current_state, batch_size):
        self.batch_size = batch_size
        self.labels = labels
        self.img_file_index = image_file_index
        self.current_state = current_state
        self.len = len(self.img_file_index) // self.batch_size
        print("for DataGenSequence", current_state, "total rows are:", len(self.img_file_index), ", len is", self.len)

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        print("loading data segmentation", idx)
        # make sure each batch size has the same amount of data
        current_batch = self.img_file_index[idx * self.batch_size: (idx + 1) * self.batch_size]
        X = np.empty((self.batch_size, resized_height, resized_width, num_channel))
        y = np.empty((self.batch_size, num_classes))

        for i, image_name in enumerate(current_batch):
            path = os.path.join(nih_chest_xray_data_dir, image_name)

            # loading data
            img = cv2.resize(cv2.imread(path), (resized_height, resized_width)).astype(np.float32)
            X[i, :, :, :] = img
            y[i, :] = labels[image_name]
           
            # only do random flipping in training status
        if self.current_state == 'train':
            # this is different from the training code
            x_augmented = X
        else:
            x_augmented = X
        
        return x_augmented, y

if __name__ == "__main__":
    prj_consts = azure_chestxray_utils.chestxray_consts()

    #Organize directories
    data_base_input_dir=os.path.join(base_dir, 
                                     os.path.join(*(prj_consts.BASE_INPUT_DIR_list)))
    data_base_output_dir=os.path.join(base_dir, 
                                      os.path.join(*(prj_consts.BASE_OUTPUT_DIR_list)))

    weights_dir = os.path.join(data_base_output_dir, os.path.join(*(prj_consts.MODEL_WEIGHTS_DIR_list))) 
    fully_trained_weights_dir = os.path.join(data_base_output_dir, os.path.join(*(prj_consts.FULLY_PRETRAINED_MODEL_DIR_list))) 

    nih_chest_xray_data_dir=os.path.join(data_base_input_dir, 
                                         os.path.join(*(prj_consts.ChestXray_IMAGES_DIR_list)))

    data_partitions_dir=os.path.join(data_base_output_dir, 
                                    os.path.join(*(prj_consts.DATA_PARTITIONS_DIR_list)))  
    label_path = os.path.join(data_partitions_dir,'labels14_unormalized_cleaned.pickle')
    partition_path = os.path.join(data_partitions_dir, 'partition14_unormalized_cleaned.pickle')

#     models_file_name= [os.path.join(weights_dir, 
#                                     'azure_chest_xray_14_weights_712split_epoch_300_val_loss_361.7687.hdf5')] 
    models_file_name= [os.path.join(fully_trained_weights_dir, 
                                   'azure_chest_xray_14_weights_712split_epoch_250_val_loss_179-4776.hdf5')] #EDIT THIS ACCORDINGLY

    num_gpu = get_available_gpus()
    # get number of available GPUs
    print("num of GPUs:", len(get_available_gpus()))
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    
    resized_height = 224
    resized_width = 224
    num_channel = 3
    num_classes = 14
    batch_size = 512 

    pathologies_name_list = prj_consts.DISEASE_list
    pathologies_name_list

    stanford_result = [0.8094, 0.9248, 0.8638, 0.7345, 0.8676, 0.7802, 0.7680, 0.8887, 0.7901, 0.8878, 0.9371, 0.8047,
                       0.8062, 0.9164]


    with open(label_path, 'rb') as f:
        labels = pickle.load(f)

    with open(partition_path, 'rb') as f:
        partition = pickle.load(f)

    # load test data
    X_test = np.empty((len(partition['test']), 224, 224, 3), dtype=np.float32)
    y_test = np.empty((len(partition['test']) - len(partition['test']) % batch_size, 14), dtype=np.float32)

    for i, npy in enumerate(partition['test']):
        if (i < len(y_test)):
            # round to batch_size
            y_test[i, :] = labels[npy]

    print("len of result is", len(y_test))
    y_pred_list = np.empty((len(models_file_name), len(partition['test']), 14), dtype=np.float32)

    # individual models
    for index, current_model_file in enumerate(models_file_name):
        print(current_model_file)
    #     model = load_model(current_model_file)
        model = azure_chestxray_keras_utils.build_model(keras_contrib.applications.densenet.DenseNetImageNet121); model.load_weights(current_model_file)

        print('evaluation for model', current_model_file)
        # y_pred = model.predict(X_test)

        y_pred = model.predict_generator(generator=DataGenSequence(labels, partition['test'], current_state='test', batch_size = batch_size),
                                         workers=32, verbose=1, max_queue_size=1)
        print("result shape", y_pred.shape)

        # add one fake row of ones in both test and pred values to avoid:
        # ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.
        y_test = np.insert(y_test, 0, np.ones((y_test.shape[1],)), 0)
        y_pred = np.insert(y_pred, 0, np.ones((y_pred.shape[1],)), 0)

        df = pd.DataFrame(columns=['Disease', 'Our AUC Score', 'Stanford AUC Score'])
        scores_list = []
        for d in range(14):
            auc = metrics.roc_auc_score(y_test[:, d], y_pred[:, d])
            df.loc[d] = [pathologies_name_list[d],
                         auc,
                         stanford_result[d]]
            scores_list.append(auc)
        
        df['Delta'] = df['Stanford AUC Score'] - df['Our AUC Score']
        df.to_csv(current_model_file + ".csv", index=False)
        print(df)


Overwriting ./xray_scripts/eval.py


In [21]:
from azureml.core.runconfig import RunConfiguration, DataReferenceConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE


run_config = RunConfiguration(framework = "python")
run_config.target = gpu_cluster_name
run_config.environment.docker.enabled = True

run_config.environment.docker.base_image = 'kateyuan/chestxraynoaml:1.0.4' #DEFAULT_CPU_IMAGE #'nvidia/cuda:9.0-cudnn7-devel' 

run_config.environment.python.user_managed_dependencies = True
run_config.environment.python.interpreter_path = '/opt/conda/bin/python'


In [23]:
from azureml.train.estimator import Estimator

script_params = {
    '--data-folder': ds.as_mount(),
}


est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target=gpu_cluster,
                entry_script='eval.py',
                environment_definition=run_config.environment)




In [24]:
%%time
from azureml.core import Run
run = exp.submit(est)
run.wait_for_completion(show_output = True)
run.get_portal_url()

RunId: chestxray-evaluate_1548950647840

Streaming azureml-logs/60_control_log.txt

Streaming log file azureml-logs/60_control_log.txt
Streaming log file azureml-logs/80_driver_log.txt

Streaming azureml-logs/80_driver_log.txt

Using TensorFlow backend.
Namespace(data_folder='/mnt/batch/tasks/shared/LS_root/jobs/sdk-chest-xray/azureml/chestxray-evaluate_1548950647840/mounts/chestxraydatastore')
Data folder is at: /mnt/batch/tasks/shared/LS_root/jobs/sdk-chest-xray/azureml/chestxray-evaluate_1548950647840/mounts/chestxraydatastore
2019-01-31 16:08:37.222315: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-01-31 16:08:38.039587: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1432] Found device 0 with properties: 
name: Tesla V100-PCIE-16GB major: 7 minor: 0 memoryClockRate(GHz): 1.38
pciBusID: 6e93:00:00.0
totalMemory: 15.78GiB freeMemory: 15.37GiB
2019-01-31 16:08:38.276414: I tens

evaluation for model /mnt/batch/tasks/shared/LS_root/jobs/sdk-chest-xray/azureml/chestxray-evaluate_1548950647840/mounts/chestxraydatastore/data/chestxray/output/fully_trained_models/azure_chest_xray_14_weights_712split_epoch_250_val_loss_179-4776.hdf5
for DataGenSequence test total rows are: 32893 , len is 64
loading data segmentation 0
loading data segmentation 1
loading data segmentation 2

 1/64 [..............................] - ETA: 25:50loading data segmentation 3

 2/64 [..............................] - ETA: 13:08loading data segmentation 4

 3/64 [>.............................] - ETA: 8:54 loading data segmentation 5

 4/64 [>.............................] - ETA: 9:21loading data segmentation 6

 5/64 [=>............................] - ETA: 7:30loading data segmentation 7

 6/64 [=>............................] - ETA: 6:18loading data segmentation 8

 7/64 [==>...........................] - ETA: 6:47loading data segmentation 9

 8/64 [==>...........................] - ETA: 5

CPU times: user 8.35 s, sys: 461 ms, total: 8.81 s
Wall time: 9min 50s
